# Lab 4: Information Extraction


**Exercise 1.** Implement a trie that allows the operations *insert* and *find*. Add all the entities from the file *winners.txt* to the trie. Using this trie, perform entity recognition in the text file *example-nobel.txt*. The files are on Moodle.

You can use as starting point the following code: 

Lien de la ressource pour les Regex : https://docs.python.org/3/howto/regex.html

In [8]:
class TrieNode:
	"""A node in the trie structure"""

	def __init__(self, char):
		# the character stored in this node
		self.char = char

		# whether this can be the end of a word
		self.is_end = False

		# a dictionary of child nodes
		# keys are characters, values are nodes
		self.children = {}


class Trie(object):
	"""The trie object"""

	def __init__(self):
		"""
		The trie has at least the root node.
		The root node does not store any character
		"""
		self.root = TrieNode("")

	def insert(self, word: str):
		"""Insert a word into the trie"""

		# TO COMPLETE
		node = self.root

		#On avance au fur et a mesure
		for i in range(len(word)):
			
			if word[i] in node.children:
				node = node.children[word[i]]
			else:
				child = TrieNode(word[i])
				node.children[word[i]] = child
				node = child

		node.is_end = True	#Le dernier noeud termine forcement

			
	def find(self, x):
		"""Return a tuple of two boolean values:
		  - first value: true or false if the prefix is present or not in the trie.
		  - second value: true if the prefix is a word in the trie
		  Example:
			  Suppose Adams is in our trie t.
			  t.find("Adams") returns (True, True)
			  t.find("Adam") returns (True, False)
		  """
		
		# TO COMPLETE
		word = x
		val_1, val_2 = False, False
		node = self.root
		for i in range(len(word)):
			if word[i] in node.children:
				node = node.children[word[i]]
			else:
				return False, False
		
		return True, node.is_end

**Exercise 2.** Supposing that all entities are written such that they start with a capital letter, perform entity recognition in the text file *example-nobel.txt* using regular expression. You should also be able to capture with the regex that an entity can have 2 or more names, such as Barack Obama. For this, first follow this tutorial https://www.w3schools.com/python/python_regex.asp on the **re library** in Python. When you have an idea for a regex, you can test it using this website https://regex101.com/ . Note the website even generates the corresponding Python code for you! You might need to read a bit about repeating a group https://www.regular-expressions.info/captureall.html .

In [46]:
import re

# Solution exercise 2
def build_trie_from_file(file_path):
    """
    Reads a file and returns a trie containing all the words in the file.
    Each word is assumed to be on a separate line.
    """
    trie = Trie()
    with open(file_path, 'r') as file:
        for line in file:
            clean_line = line.strip()
            ''' '''
            #Utile si on ne veut qu'une petite partie du code
            words = clean_line.split() 
            for word in words:
                trie.insert(word)
            

            #Si on veut garder les mots composes
            trie.insert(clean_line)
    return trie

def track_with_trie(file_train, file_tracked = "./data/example-nobel.txt"):
    list = []
    trie = build_trie_from_file(file_train)
    #On cherche maintenant les noms propres du fichier example-nobel.txt
    file = open(file_tracked, 'r')
    for line in file:
        words = line.strip().split()
        i = 0
        while i < len(words):
            word = words[i]
            #print(f'{i} : {word}') #Used for debug
            a, b = trie.find(word)
            longest_word = None
            next_i = i+1
            #Si notre mot est un npm propre
            if b:
                longest_word = word
            if a:
                j = i+1
                while a:
                    a, b = trie.find(" ".join(words[i:j+1]))
                    #print(f'long : {longest_word}, thing : {" ".join(words[i:j+1])}, a : {a}, b: {b}')  #Used for debug
                    if b:
                        longest_word = " ".join(words[i:j+1])
                        next_i = j+1
                    j+=1
            if longest_word != None:
                list.append(longest_word)

            i = next_i
    return list


print(track_with_trie("./data/winners.txt"))

def track_with_re(file_tracked):
    file = open(file_tracked, 'r')
    list = []
    for line in file:
        x = re.findall("[A-Z][a-z]+ [A-Z][a-z]+", line.strip())
        #x = re.findall("[A-Z][a-z]+ ", line.strip())
        list += x
    return list

print(track_with_re("./data/example-nobel.txt")) 

['Henry Kissinger', 'Le', 'Duc', 'of', 'Kissinger', 'North', 'North', 'Kissinger', 'North', 'of', 'Yasser', 'Shimon', 'Yitzhak Rabin', 'of', 'Arafat', 'Arafat', 'Barack Obama', 'Obama', 'of', 'Obama', 'of', 'of', 'Obama', 'Jimmy Carter', 'Al', 'of']
['Nobel Peace', 'Henry Kissinger', 'Le Duc', 'Norwegian Nobel', 'North Vietnam', 'United States', 'Yasser Arafat', 'Shimon Peres', 'Yitzhak Rabin', 'Peace Prize', 'Norwegian Nobel', 'Peace Prize', 'Barack Obama', 'United States', 'Past Peace', 'Peace Prizes', 'Jimmy Carter', 'Al Gore']


**Exercise 3.** First install spaCy: https://spacy.io/usage  using the link we provided, according to your operating system. Perform entity recognition using spaCy https://spacy.io/usage/linguistic-features#named-entities on the file example-nobel.txt . Compute the F1 score of the techniques from Exercise 1, 2 and of spaCy on the task of finding entities in the file example-nobel.txt. Note that for this you need to create the gold standard, i.e. the correct annotations. Do not take into account the entities of type DATE returned by spaCy.  

In [48]:
# Solution exercise 3
import spacy

file_name = "./data/example-nobel.txt"

def spacy_track(file_name):
    file = open(file_name, 'r')
    for line in file:
        nlp = spacy.load("en_core_web_sm")
        doc = nlp(line)

        for ent in doc.ents:
            if ent.label_ != "DATE":
                print(ent.text, ent.start_char, ent.end_char, ent.label_)
#spacy_track(file_name)

def f_1():
    file_tracked = "./data/example-nobel.txt"
    file = open(file_tracked, 'r')
    for line in file:
        #Method 1
        words_1 = track_with_trie(file_tracked) 
        
        #Method 2
        words_2 = re.findall("[A-Z][a-z]+ [A-Z][a-z]+", line.strip())
        
        #Method 3
        nlp = spacy.load("en_core_web_sm")
        doc = nlp(line)
        words_3 = []
        for ent in doc.ents:
            if ent.label_ != "DATE":
                words_3.append(ent.text)

        #Ensuite pour le recall on regarde a quel point les words_i matchent avec words_abs avec la liste absolue des noms propres dans le fichier
        


**Exercise 4.** spaCy does not have a relation extraction module! What can we do? Let's implement our own using the ReVerb algorithm that we saw during the class. In order to find the parts of speech needed to the algorithm, check this https://spacy.io/usage/linguistic-features#pos-tagging . Note that spaCy allows you to add regular expressions easily, such that you can implement ReVerb https://spacy.io/usage/rule-based-matching . Make sure to first split the text in sentences such that you can correctly identify the subject and object, by adapting the following code:

In [73]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("sentencizer")
doc = nlp("Paris is the capital of France. Paris is located in France.")
for sentence in doc.sents:
    print(sentence)
    #print(type(sentence))
print("Noun phrases:", [(chunk.text, chunk.start, chunk.end) for chunk in
                                doc.noun_chunks])

Paris is the capital of France.
Paris is located in France.
Noun phrases: [('Paris', 0, 1), ('the capital', 2, 4), ('France', 5, 6), ('Paris', 7, 8), ('France', 11, 12)]


In [ ]:
'''
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)'
            '''


Here is an example of the first part of the regular expression you need to write to get the predicate:

In [63]:
'''
Funny : this works
result = "{} {}".format(string1, string2)  # Format placeholders
'''

'\nFunny : this works\nresult = "{} {}".format(string1, string2)  # Format placeholders\n'

In [67]:
def read_file(file_path):
    txt = ""
    file = open(file_path, 'r')
    for line in file:
        txt += line.strip()
    return txt

''''
text = read_file("./arthur.txt")
print(text)
'''


'\'\ntext = read_file("./arthur.txt")\nprint(text)\n'

In [ ]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

#V
pattern1 = [{"POS": {"IN":["AUX","VERB"]}}, {"POS": "VERB", "OP":"?"}, 
            {"POS": "PART", "OP":"?"}, {"POS": "ADV", "OP":"?"}]
matcher.add("V", [pattern1], greedy='LONGEST')

#VW*P
pattern2 = [{"POS": {"IN":["AUX","VERB"]}}, {"POS": "VERB", "OP":"?"}, 
            {"POS": "PART", "OP":"?"}, {"POS": "ADV", "OP":"?"},
            
            {"POS": {"IN":["NOUN","ADJ","ADV","PRON","DET"]}, "OP":"*"},

            {"POS": {"IN":["ADP","PART"]}}]
matcher.add("VW*P", [pattern2], greedy='LONGEST')

#VP
pattern3 = [{"POS": {"IN":["AUX","VERB"]}}, {"POS": "VERB", "OP":"?"}, 
            {"POS": "PART", "OP":"?"}, {"POS": "ADV", "OP":"?"},
            {"POS": {"IN":["ADP","PART"]}}]
matcher.add("VP", [pattern3], greedy='LONGEST')

text = read_file("./data/simplewiki.txt")
text = "Paris is the capital of France."
#print(text)

doc = nlp(text)
for token in doc:
    print(token.text, token.pos_)
matches = matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(string_id, start, end, span.text)

Paris PROPN
is AUX
the DET
capital NOUN
of ADP
France PROPN
. PUNCT
V 1 2 is
VW*P 1 5 is the capital of


Finish the code by adding the full regular expression from the class, V|VP|VW*P.
First try your code on a small paragraph, e.g. on the sentences above you get the triples (Paris, is the capital of, France) and (Paris, is located in , France). Then test your algorithm on a sample from Wikipedia, contained in the file *simplewiki.txt*.

Note that you can use part-of-speech tags to improve your NER algorithm that uses regular expression on capital letter (Exercise 2). Filter out words that are not nouns from your matches. 

In [ ]:
# Solution exercise 4

**Exercise 5.** Filter the triples you extracted for the sample of Wikipedia such that you keep only the ones for which the subject is a named entity. 
You might have several mentions of the same entity, perform deduplication on these mentions (i.e. determine which mentions refer to the same object). For this, write a simple algorithm that given two entities in input, computes if they are the same using their names, and the label of the entities that is given when you run the named entity function of spaCy. If you find that mentions of entities with the same name that refer to different objects, modify the mention's name such that the mentions are distinct (eg: Europe_continent, Europe_person).  
Create a graph in NetworkX using the triples your obtain and find the important nodes in this graph, using different measures of importance that we have seen during the first course.

In [ ]:
# Solution exercise 5